In [13]:
import functools
import os
import geopandas as gpd
import json

from pathlib import Path
from traitlets import (
    List, Any, link, observe, Unicode, HasTraits
)
import ipyvuetify as v
import sepal_ui.sepalwidgets as sw

In [14]:
from sepal_ui.scripts import utils as su
from sepal_ui.mapping import mapping as m

In [15]:
def alert_error(alert):
    """Decorator to execute try/except sentence
    and toggle loading button object
    
    Params:
        alert (sw.Alert): Alert to display errors
    """
    def decorator_alert_error(func):
        @functools.wraps(func)
        def wrapper_alert_error(*args, **kwargs):
            try:
                value = func(*args, **kwargs)
            except Exception as e:
                alert.add_msg(f'{e}', type_='error')
                raise e
            return value
        return wrapper_alert_error
    return decorator_alert_error


In [16]:
class Flex(v.Flex, sw.SepalWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [17]:
class Select(v.Select, sw.SepalWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [18]:
ALL = 'All'

In [31]:
class ColumnField(v.Flex, sw.SepalWidget):
    
    column_items = List([]).tag(sync=True)
    field_items = List([]).tag(sync=True)
    
    ALL_ITEMS = [{'text':'Use all features', 'value':ALL}, {'divider':True}]
    
    def __init__(self,*args,**kwargs):
        
        super().__init__(*args,**kwargs)
        
        self.w_column = Select(
            _metadata={'name':'column'},
            label="Filter by column",
            v_model=ALL,
            items=self.column_items
        )
        
        self.w_field = Select(
            _metadata={'name':'field'},
            label="Select field",
            v_model="",
            items=self.field_items
        ).hide()
        
        self.children=[
            self.w_column,
            self.w_field
        ]
        
        # Link traits
        link((self, 'field_items'),(self.w_field, 'items'))
        
        # Events
        
        self.w_column.observe(self.toggle_fields, 'v_model')
    
    def toggle_fields(self, change):
        """Toggle field widget"""
        self.w_field.show() if change['new'] != ALL else self.w_field.hide()
        
    @observe('column_items')
    def _add_all_item(self, change):
        """Add 'All' item to the columns items as the first option"""
        self.w_column.items =  self.ALL_ITEMS + change['new']
        
    def reset(self):
        """Reset items to its original state"""
        self.column_items = self.field_items = []

In [32]:
class AoiView(v.Card):
    
    method = Unicode('').tag(sync=True)
    column = Any('').tag(sync=True)
    field = Any('').tag(sync=True)
    
    def __init__(self, model, map_, *args, **kwargs):
        
        self.map_=map_
        self.methods = self._get_methods()
        self.column_field = ColumnField()
        
        self.alert = sw.Alert()
        self.model = model(self.alert)
        
        w_method = v.Select(
            label = 'Select a method',
            v_model = self.method,
            items = self._get_methods()
        )
        
        self.w_countries = v.Select(
            label="Select country",
            v_model='',
            items=self._get_countries(),
        )
        self.btn_country = sw.Btn('Select', small=True)
        
        w_countries_btn = Flex(
            class_='d-flex align-center mb-2',
            row=True, 
            children=[self.w_countries, self.btn_country])
        
        
        self.w_file = sw.FileInput(['.shp'], '/home/dguerrero/restoration_viewer/shp/')
        
        self.btn_file = sw.Btn('Select file', small=True)
        w_file_btn = Flex(
            class_='d-flex align-center mb-2',
            row=True, 
            children=[self.w_file, self.btn_file])
        
        self.components = {
            'Country' : w_countries_btn,
            'Upload file' : w_file_btn,
            'Column_field' : self.column_field,
        }
        
        self._hide_components()
        
        super().__init__(*args, **kwargs)
        
        
        # Link traits view
        link((self, 'method'),(w_method, 'v_model'))
        link((self, 'column'),(self.column_field.w_column, 'v_model'))
        link((self, 'field'),(self.column_field.w_field, 'v_model'))
        
        # Link traits with model
        link((self.model, 'country'),(self.w_countries, 'v_model'))
        
        # Events
        self.btn_file.on_event('click', self._file_btn_event)
        
        
        self.children=[
            self.alert,
            w_method,
            w_countries_btn,
            w_file_btn,
            self.column_field,
        ]
            
    def _file_btn_event(self, widget, event, data):
        """Define behavior when the file button is clicked"""
        
        self.column_field.reset()
        
        # Create a geopandas dataset
        self.model.shape_to_gpd(self.w_file.file)
        
        # Populate columns widget with all columsn plus 'ALL' in case user wants to use all 
        self.column_field.column_items = self.model._get_columns()
        
        # Show column-field widget
        self.column_field.show()
    
    @observe('column')
    def _get_fields(self, change):
        """Populate widget field items with fields"""
        
        # Reset fields items
        self.column_field.items = []
        
        if self.column == ALL:
            "All geometries were selected"
        else:
            self.column_field.w_field.loading=True
            self.column_field.field_items = self.model._get_fields(self.column)
            self.column_field.w_field.loading=False
    
    @observe('field')
    def _get_selected_feature(self, change):
        """Define selected feature with the current options"""

        self.model.selected_feature = self.model._get_selected(self.column, self.field)

    def _hide_components(self):
        """Hide all possible componentes"""
        
        for component in self.components.values():
            su.hide_component(component)
        
    def _get_methods(self):
        """Handle which methods will be displayed in select widget"""
        
        return ['Draw on map', 'Country', 'Upload file']
    
    def _get_countries(self):
        """Create a list of countries"""
        
        return list(range(10))
    
    @observe('method')
    def _aoi_method_event(self, change):
        
        method = change['new']
        self._hide_components()
        
        if method == 'Draw on map':
            self.map_.show_dc()
        else:
            self.map_.hide_dc()
            su.show_component(self.components[method])

In [33]:
class AoiModel(HasTraits):
    
    country = Any('').tag(sync=True)

    def __init__(self, alert, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.alert = alert
        self.gpd = None
        self.selected_feature = None
        
        # Decorate methods
        self.shape_to_gpd = alert_error(self.alert)(self.shape_to_gpd)
            
    def shape_to_gpd(self, file):
        """ Converts shapefile into geopandas"""
        
        file_path = Path(file)
        
        assert file_path.exists(), "File doesn't exists"
        
        if file_path.suffix == '.shp':

            self.gdf = gpd.read_file(str(file_path))
            self.gdf = self.gdf.to_crs("EPSG:4326")
            
    def _get_columns(self):
        """Return all columns skiping geometry"""
        return list(set(['geometry'])^set(self.gdf.columns.to_list()))
        
    def _get_fields(self, column):
        """Return fields from selected column"""
        return self.gdf[column].to_list()
    
    def _get_selected(self, column, field):
        """Get selected element"""
        
        return self.gdf[self.gdf[column] == field]
        

In [34]:
map_=m.SepalMap()

In [35]:
aoi_view = AoiView(map_=map_, model = AoiModel)

In [36]:
aoi_view

AoiView(children=[Alert(class_='d-none', text=True, type='info'), Select(items=['Draw on map', 'Country', 'Upl…

In [37]:
map_

SepalMap(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…